In [18]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [19]:
##---------------DATOS DE ENTRENAMIENTO -------------------------##
##Cargar los datos con extención .mat
from scipy.io import loadmat
x = loadmat('/DatosEntrenamiento2.mat')
##Traernos los datos del archivo .mat que nos interesan
y = x['data_te']
##Convertir los datos en un array de Numpy
z = np.array(y)
##Convertir el Array en un DataFrame de Pandas
df = pd.DataFrame(z)
##Contar el numero de datos nulos del dataFrame
nulos = np.count_nonzero(pd.isnull(df) == True)
print(nulos)
##Nombrar las Columnas
df.columns = ['uplink_size_mean','uplink_size_median','uplink_size_MAD','uplink_size_STD','uplink_size_Skewness','uplink_size_Kurtosis','uplink_size_MAX','uplink_size_MIN','uplink_size_MeanSquare','downlink_size_mean','downlink_size_median','downlink_size_MAD', 'downlink_size_STD','downlink_size_Skewness','downlink_size_Kurtosis','downlink_size_MAX','downlink_size_MIN','downlink_size_MeanSquare','both_links_size_mean','both_links_size_mean','both_links_size_MAD','both_links_size_STD','both_links_size_Skewness','both_links_size_Kurtosis','both_links_size_MAX','both_links_size_MIN','both_links_size_MeanSquare','uplink_interval_mean','uplink_interval_median','uplink_interval_MAD','uplink_interval_STD','uplink_interval_Skewness','uplink_interval_Kurtosis','uplink_interval_MAX','uplink_interval_MIN','uplink_interval_MeanSquare','downlink_interval_mean','downlink_interval_median','downlink_interval_MAD','downlink_interval_STD','downlink_interval_Skewness','downlink_interval_Kurtosis','downlink_interval_MAX','downlink_interval_MIN','downlink_interval_MeanSquare','both_links_interval_mean','both_links_interval_median','both_links_interval_MAD','both_links_interval_STD','both_links_interval_Skewness','both_links_interval_Kurtosis','both_links_interval_MAX','both_links_interval_MIN','both_links_interval_MeanSquare','Label']
df.sample(8)

0


,uplink_size_mean,uplink_size_median,uplink_size_MAD,uplink_size_STD,uplink_size_Skewness,uplink_size_Kurtosis,uplink_size_MAX,uplink_size_MIN,uplink_size_MeanSquare,downlink_size_mean,downlink_size_median,downlink_size_MAD,downlink_size_STD,downlink_size_Skewness,downlink_size_Kurtosis,downlink_size_MAX,downlink_size_MIN,downlink_size_MeanSquare,both_links_size_mean,both_links_size_mean,both_links_size_MAD,both_links_size_STD,both_links_size_Skewness,both_links_size_Kurtosis,both_links_size_MAX,both_links_size_MIN,both_links_size_MeanSquare,uplink_interval_mean,uplink_interval_median,uplink_interval_MAD,uplink_interval_STD,uplink_interval_Skewness,uplink_interval_Kurtosis,uplink_interval_MAX,uplink_interval_MIN,uplink_interval_MeanSquare,downlink_interval_mean,downlink_interval_median,downlink_interval_MAD,downlink_interval_STD,downlink_interval_Skewness,downlink_interval_Kurtosis,downlink_interval_MAX,downlink_interval_MIN,downlink_interval_MeanSquare,both_links_interval_mean,both_links_interval_median,both_links_interval_MAD,both_links_interval_STD,both_links_interval_Skewness,both_links_interval_Kurtosis,both_links_interval_MAX,both_links_interval_MIN,both_links_interval_MeanSquare,Label
13391,0.016736,0.038707,0.003472,0.004246,2.815011,6.671278,0.164684,0.000115,0.041674,241.277800,55.275210,228.0,0.0000,5.417310,28.486280,562.0,228.0,247.356900,0.007051,0.021852,0.001113,0.001069,4.424458,19.475790,0.123615,0.000331,0.022798,1635.922000,126.725800,1676.0,0.0000,-2.797198,6.028939,1676.0,1224.0,1640.746000,0.006029,0.020014,0.000816,0.000819,4.935146,24.068060,0.122688,0.000105,0.020806,1133.85,681.147400,1676.0,0.0000,-0.525068,-1.686806,1676.0,228.0,1320.961000,0.0
7258,0.007931,0.008481,0.004861,0.007175,0.492610,1.611679,0.022103,0.000013,0.011517,116.363636,50.947165,88.0,17.7912,0.745890,1.687215,198.0,76.0,126.718012,0.003297,0.006189,0.000265,0.000361,1.843991,4.969727,0.022136,0.000006,0.006972,737.313433,702.486368,417.0,505.5666,0.287557,1.231637,1610.0,76.0,1014.767321,0.002617,0.005738,0.000028,0.000029,2.266125,6.836581,0.022092,0.000006,0.006280,532.40,644.936392,88.0,17.7912,0.932923,2.019677,1610.0,76.0,833.806500,1.0
1749,0.015733,0.032728,0.001805,0.002673,2.567613,8.305081,0.117000,0.000001,0.035693,124.333333,41.037275,88.0,2.9652,0.166382,1.028856,168.0,86.0,130.662415,0.005008,0.020868,0.000060,0.000086,5.375017,31.461314,0.128909,0.000002,0.021327,930.973684,597.673422,1172.0,471.4668,-0.583045,1.585404,1578.0,76.0,1104.185375,0.003806,0.016950,0.000029,0.000039,6.088742,40.472371,0.117000,0.000001,0.017289,737.38,625.210620,906.5,989.6355,0.051358,1.161849,1578.0,76.0,964.732450,1.0
12304,0.003333,0.007908,0.000117,0.000047,2.221756,3.585159,0.028546,0.000055,0.008451,233.321400,15.318960,228.0,0.0000,3.114584,9.130745,297.0,228.0,233.805900,0.001489,0.004809,0.000496,0.000236,6.687155,47.012230,0.038567,0.000328,0.005002,1656.917000,92.162710,1676.0,0.0000,-4.492081,18.435410,1676.0,1218.0,1659.442000,0.001072,0.003115,0.000382,0.000368,6.251148,42.232840,0.025921,0.000055,0.003280,1258.31,647.186100,1676.0,0.0000,-0.927537,-1.116439,1676.0,228.0,1413.508000,0.0
11474,0.052778,0.098275,0.004797,0.006961,2.063911,3.481395,0.382638,0.000090,0.109652,527.565200,448.422900,216.0,0.0000,0.757839,-1.441301,1215.0,216.0,686.050600,0.015793,0.047614,0.000488,0.000387,6.008107,42.203040,0.381030,0.000116,0.049870,1167.857000,621.287300,1676.0,0.0000,-0.517558,-1.593092,1676.0,222.0,1320.937000,0.012705,0.042026,0.000504,0.000541,6.918619,56.245950,0.380887,0.000090,0.043703,1020.59,643.697500,1181.0,733.8870,-0.152913,-1.804454,1676.0,216.0,1204.910000,0.0
8783,0.025130,0.033682,0.002345,0.003302,1.014445,-0.213958,0.122007,0.000118,0.041747,743.632653,647.190392,259.0,54.8562,0.568814,-1.591220,1676.0,222.0,981.476876,0.025381,0.032625,0.002379,0.003324,0.865342,-0.476507,0.122546,0.000098,0.041082,298.725490,153.597276,216.0,0.0000,2.009134,2.836164,734.0,216.0,335.211054,0.012949,0.025337,0.000630,0.0

In [20]:
YEntrenamiento = df['Label']
XEntrenamiento = scaler.fit_transform(df.drop(['Label'],axis=1))

In [33]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.svm import SVC
import time
from sklearn.metrics import accuracy_score

In [21]:
def entrenamiento_sin_seleccion_caracteristicas(splits, X, Y):
    """
    Función que ejecuta el entrenamiento del modelo sin una selección particular
    de las características

      Parámetros:splits : numero de particiones  a realizar
      Retorna:
      1. El modelo entreando
      2. El vector de errores
      3. El Intervalo de confianza
      4. El tiempo de procesamiento
    """
    #Implemetamos la metodología de validación
    Errores = np.ones(splits)
    Score = np.ones(splits)
    times = np.ones(splits)
    j = 0
    kf = KFold(n_splits=splits)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        #Creamos el clasificador SVM.
        clf = SVC(kernel="linear", C=1)
        #Aquí se entran y se valida el modelo sin hacer selección de características
        tiempo_i = time.time()
        clf.fit(X_train,y_train)
        # Validación del modelo
        Errores[j] = accuracy_score(y_true=y_test, y_pred=clf.predict(X_test))
        times[j] = time.time()-tiempo_i
        j+=1

    return np.mean(Errores), np.std(Errores), np.mean(times)

In [22]:
from sklearn.decomposition import PCA
def entrenamiento_pca_ext_caracteristicas(n_comp, X, Y):
    """
    Esta función realiza la reducción de la dimensionalidad sobre el conjunto de
    datos de entrenamiento, de acuerdo con las particiones especificadas usando PCA

    Parámetros:
    n_comp, int, Número de componentes para reducción
    n_sets,int, Número de particiones
    X: numpy Array de características
    Y: numpy Array  Vector de etiquetas

    Retorna: 
    ErrorValidacion: El valor medio de errores
    IC_Validacion: Intervalo de confianza
    T_ejec:  El  valor medio del tiempo de ejecución
    """  
    #Implemetamos la metodología de validación 
    Errores = np.ones(5)
    times = np.ones(5)
    j = 0
    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X):  
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        
        # ¿es necesario estandarizacion de datos?
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
 
        #dejar el mismo nombre del objeto 
        pca = PCA(n_components=n_comp)
        # para calcular costo computacional
        tiempo_i = time.time()
        # es recomendable usar el metodo que ajusta y transforma
        X_train_pca = pca.fit_transform(X_train, y_train)
        # aca solo usar el metodo de transformar (ya que en el anterior el pca se ajusto)
        X_test_pca = pca.transform(X_test)
        # entrenar el modelo usando las caractieristicas transformadas por PCA
        clf = SVC(kernel="linear", C=1)
        clf.fit(X=X_train_pca, y=y_train)
        tiempo_o = time.time()-tiempo_i
        Errores[j] = accuracy_score(y_true=y_test, y_pred=clf.predict(X_test_pca))
        times[j] = tiempo_o
        j+=1


    return np.mean(Errores), np.std(Errores), np.mean(times)

In [23]:
def experimentar_PCA(n_feats, X, Y):
    """
    Esta función realiza la comparación del desempeño de RFE utilizando diferente 
    número de feats y particionando el conjunto de datos en diferente número de 
    subconjuntos

    Parámetros:
    X (numpy.array), El arreglo numpy de características
    Y (numpy.array), El vector de etiquetas
    n_feats, Vector de números enteros que indica el número de características
              que debe utilizar el modelo
    n_sets, Vector de números enteros que indica el número de particiones

    Retorna:  
    - DataFrame con las columnas: CON_SEL, NUM_VAR, ERROR_VALIDACION, IC_STD_VALIDACION, 
    y T_EJECUCION. 

    """
    df = pd.DataFrame()
    idx = 0
    split_number = 5
    #Sin selección de características
    error,ic_error,t_ex = entrenamiento_sin_seleccion_caracteristicas(split_number, X,Y)  
    df.loc[idx,'CON_SEL'] = 'NO'
    df.loc[idx,'NUM_VAR'] = X.shape[1] # se usan todas las caracteristicas
    df.loc[idx,'ERROR_VALIDACION'] = error
    df.loc[idx,'IC_STD_VALIDACION'] = ic_error
    df.loc[idx,'T_EJECUCION'] = t_ex
    idx+=1
    print("termina experimento sin selección")
    #Con selección de características
    
    for f in n_feats:
        #Implemetamos la metodología de validación 
        error, ic_error, t_ex = entrenamiento_pca_ext_caracteristicas(n_comp=f, X=X,Y=Y)
        df.loc[idx,'CON_SEL'] = 'SI'
        df.loc[idx,'NUM_VAR'] = X.shape[1]
        df.loc[idx,'ERROR_VALIDACION'] = error
        df.loc[idx, 'IC_STD_VALIDACION'] = ic_error
        df.loc[idx, 'T_EJECUCION'] = t_ex
        idx+=1
    return df

In [34]:
experimentar_PCA(n_feats=[2,5,10,15,20], X= XEntrenamiento, Y = YEntrenamiento)

termina experimento sin selección


,CON_SEL,NUM_VAR,ERROR_VALIDACION,IC_STD_VALIDACION,T_EJECUCION
0,NO,54.0,1.000000,0.000000,0.086281
1,SI,54.0,0.843309,0.089130,1.656519
2,SI,54.0,0.999872,0.000255,0.172225
3,SI,54.0,0.999936,0.000128,0.159036
4,SI,54.0,0.999809,0.000255,0.189754
5,SI,54.0,0.999872,0.000255,0.227863
